# A la maniere de 'continuum' (Gyorgi Ligeti)

Generates a texture similar to Ligeti's Continuum  using stocastic processes controlled by line envelopes.

Running this notebook requires a jupyter kernel that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px; color: grey;">

Python imports:

In [ ]:
from musx import version, Score, Note, Seq, MidiFile, Shuffle, Choose, scale, setmidiplayer, playfile
from musx.midi.gm import Harpsichord
print(f"musx version: {version}")

Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")

The `register()` generator composes the notes for a section of music:

In [1]:
def register (score, rhy, dur, low, high, amp):
    """
    Creates a chromatic scale between low and high, notes in the scale
    are shuffled to procduce a saturated texture.
    """ 
    pat = Shuffle(scale(low, high-low+1, 1))
    while score.elapsed < dur:
        keyn = pat.next()
        note = Note(time=score.now, duration=rhy, pitch=keyn, amplitude=amp)
        score.add(note)
        yield rhy

print(f"register: {register}")

register: <function register at 0x1038d2680>


The `continuum()` generator calls `register()` to create successive sections of the composition:

In [ ]:
def continuum (score, rhy, minkeys, maxkeys, seclens):
    """
    Calls register() to create the next section's material, then
    waits until that section is over before creating another section.

    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    rhy : int | float
        The amount of time to wait between this note and the next.
    minkeys : list
        A list of keynums defining the lowest possible note for each call to register().
    maxkeys : list
        A list of keynums defining the highest possible note for each call to register().
    seclens : list
        A list of section lengths, in seconds, for the entire composition.

    """
    # random pattern of section lengths.
    pat = Choose(seclens)
    # iterate all the min and max key numbers 
    for low, high in zip(minkeys, maxkeys):
        # get the section's duration
        secdur = pat.next()
        # sprout the next section
        score.compose(register(score, rhy, secdur, low, high, .4))
        # wait till end of section
        yield secdur
        
print(f"continuum: {continuum}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack(ins={0: Harpsichord})
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1:", track1)

Create a score and pass it track1 to hold the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Lower and upper bounds on keynum choices:

In [ ]:
minkeys = [60, 59, 58, 57, 56, 55, 54, 53, 52, 
           53, 54, 55, 56, 57, 58, 59, 60, 61, 
           62, 63, 64, 65, 66, 67, 68, 69, 70,
           71, 72, 73, 74, 75, 76, 77, 78, 79, 
           80, 82, 83, 84, 85, 86, 87, 88, 89, 89]
print(f"minkeys: {minkeys}")

maxkeys = [62, 63, 64, 65, 66, 67, 68, 69, 70, 
           70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
           71, 72, 73, 74, 76, 79, 83, 86, 88, 89,
           89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89]
print(f"maxkeys: {maxkeys}")

Length of sections:

In [ ]:
seclens = [.5, 1, 1.5, 2, 2.5]
print(f"seclens: {seclens}")

Speed of rhythm:

In [ ]:
rate = .075
print(f"rate: {rate}")

Create the composition:

In [ ]:
score.compose(continuum(score, rate, minkeys, maxkeys, seclens))
print("OK!")

Write the tracks to a midi file in the current directory:

In [ ]:
file = MidiFile("continuum.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if a terminal based midi player is installed:

In [ ]:
playfile(file.pathname)

Plot the keynum range of the composition if matplotlib is installed.

In [ ]:
try:
    import matplotlib.pyplot as plt
except:
    print("Sorry, can't show you the envelopes because matplotlib is not installed /:(")
else:
    p1x,p1y = [x for x in range(len(minkeys))], [y for y in minkeys]
    plt.plot(p1x, p1y)
    p2x,p2y = [x for x in range(len(maxkeys))], [y for y in maxkeys]
    plt.plot(p2x, p2y)
    plt.show()